In [1]:
!pip install --upgrade yfinance pandas requests datetime pandas_market_calendars

  Using cached yfinance-0.2.66-py2.py3-none-any.whl.metadata (6.0 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached DateTime-5.5-py3-none-any.whl.metadata (33 kB)
  Using cached pandas_market_calendars-5.1.1-py3-none-any.whl.metadata (9.7 kB)
  Using cached multitasking-0.0.12.tar.gz (19 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached peewee-3.18.2.tar.gz (949 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached beautifulsoup4-4.14.2-py3-none-any.whl.metadata (3.8 kB)
  Using cached curl_cffi-0.13.0-cp39-abi3-win_amd64.whl.me

  DEPRECATION: Building 'multitasking' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'multitasking'. Discussion can be found at https://github.com/pypa/pip/issues/6334


변동성 구하고자 S&P의 표준편차 계산

In [2]:
import yfinance as yf

asset = "^GSPC" 
startdate = "2007-06-01"
enddate = "2009-06-30"
asset_price = yf.download(asset, start=startdate, end=enddate)
print(asset_price)

C:\Users\Dongjun\AppData\Local\Temp\ipykernel_25320\2085601013.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  asset_price = yf.download(asset, start=startdate, end=enddate)
[*********************100%***********************]  1 of 1 completed

Price             Close         High          Low         Open      Volume
Ticker            ^GSPC        ^GSPC        ^GSPC        ^GSPC       ^GSPC
Date                                                                      
2007-06-01  1536.339966  1540.560059  1530.619995  1530.619995  2927020000
2007-06-04  1539.180054  1540.530029  1532.310059  1536.280029  2738930000
2007-06-05  1530.949951  1539.119995  1525.619995  1539.119995  2939450000
2007-06-06  1517.380005  1530.569946  1514.130005  1530.569946  2964190000
2007-06-07  1490.719971  1517.359985  1490.369995  1517.359985  3538470000
...                 ...          ...          ...          ...         ...
2009-06-23   895.099976   898.690002   888.859985   893.460022  5071020000
2009-06-24   900.940002   910.849976   896.309998   896.309998  4636720000
2009-06-25   920.260010   921.419983   896.270020   899.450012  4911240000
2009-06-26   918.900024   922.000000   913.030029   918.840027  6076660000
2009-06-29   927.229980  

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime
from datetime import datetime
import pandas_market_calendars as mcal

# SNP500ELS001 클래스 선언
class SNP500ELS001():
	# 클래스 인스턴스 설정 시 아래 els 처럼 ticker를 넣어야 함
    def __init__(self, ticker: str, calendar: str):
        self.ticker = ticker
        self.cal = calendar

    def Get_Variance(self, startdate: str, enddate: str):
        # 아래 시작일과 종료일로 데이터 다운로드
        asset_price = yf.download(self.ticker, start=startdate, end=enddate)
        # 종가 데이터를 통한 일일 변화율 산출, squeeze()로 series로 변환
        asset_returns = asset_price["Close"].squeeze().pct_change().dropna()
        # 변화율 데이터로 표준편차 계산 (scaling)
        asset_vol = np.std(asset_returns)
        return asset_vol
    
    def Get_TimeUnit(self, startdate: str, enddate: str):
    	# calname 이라는 달력 인스턴스 생성
        calname = mcal.get_calendar(self.cal)
        # 문자열로 받은 시작일을 날짜형식으로 type 변경
        stdate = datetime.strptime(startdate, "%Y-%m-%d")
        # 문자열로 받은 종료일을 날짜형식으로 type 변경
        eddate = datetime.strptime(enddate, "%Y-%m-%d")
        # 뉴욕 영업일 데이터에서 시작일~ 종료일 사이의 영업일 목록을 만들기
        schedule = calname.valid_days(start_date=stdate, end_date=eddate)
        # 입수한 Timestamp 형식 영업일 데이터를 문자열형식으로 변환해서 list 데이터에 넣기
        datestrlist = [date.strftime('%Y-%m-%d') for date in schedule]
        # 문자열형식 데이터를 다시 날짜형식으로 변환해서 리스트에 저장
        datelist = [datetime.strptime(date, '%Y-%m-%d') for date in datestrlist]
        # 날짜 리스트 return
        return datelist
    
    # curvalue: 현재주가, rfr: 무위험수익률, asset_vol: 자산변동성, timeunit: 시간단위
    # 변수로 받는 함수
    def GeometricBrownianMotion(self, currentvalue: float, rfr: float, asset_vol: float, timeunit: float):
        z = np.random.randn()
        delta_t = 1/timeunit
        # 위 식 1을 그대로 코드로 나타낸 부분
        nextvalue = currentvalue * np.exp((rfr - 0.5*(asset_vol**2))*delta_t \
                    + asset_vol*np.sqrt(delta_t)*z)
        return nextvalue
    
         # 주가경로를 최종적으로 만들어내는 함수
    def Get_StochasticStockProcess(self, maxno: int, rfr: int, assetvol: float, timeunit: int, datelist):
        # 빈 데이터프레임 만들기: 단위 데이터 프레임을 합칠 전체 데이터 프레임
        totdf = pd.DataFrame()
        # 변수로 받은 시뮬레이션횟수(maxno)만큼 시나리오 만들기
        for simulationno in range(1, maxno+1):
            # 단위 데이터 프레임 컬럼 정의 "DATE", "SCENARIO&시도횟수"
            columns = ["DATE", f"SCENARIO{simulationno}"]
            # 단위 데이터 프레임 만들기
            subdf = pd.DataFrame(columns=columns)
            # 결과를 list 에 담기위한 빈 list 생성
            results = []
            # 변수로 받은 datalist마다 for 문을 통해 작업
            for date in datelist:
            	# datelist[0] 즉, 첫째날짜일 경우 basevalue 처리(최초 시작점의 가치)
                if date == datelist[0]:
                    curvalue = basevalue
                    result = [date, curvalue]
                    results.append(result)
                # datelist[0]보다 클 경우는 GeometricBrownianMotion 함수로 다음기의 주가 계산
                else:
                    nextvalue = self.GeometricBrownianMotion(curvalue, rfr, assetvol, timeunit)
                    # 주가를 GeometricBrownianMotion 함수로 구한 value로 최신화
                    curvalue = nextvalue
                    # 결과를 list화 하기
                    result = [date, curvalue]
                    # 단위 결과 list를 모아서 시나리오 한 단위의 결과 list인 results만들어가기
                    results.append(result)
            # 시나리오 한 단위의 결과인 results를 subdf 로 데이터 프레임으로 만들기
            subdf = pd.DataFrame(data=results, columns=columns)
            # 시나리오 한 단위의 결과인 단위 데이터프레임 subdf 를
            # 기존까지 작업한 데이터인 totdf 와 mergre 로 합치기
        if simulationno == 1:
            totdf = subdf
        else:
            totdf = pd.merge(totdf, subdf, on="DATE")
        return totdf
    
    
    def Early_PaymentDate(self, datelist: list, step: int, curdate: datetime):
    # step 이 0 일 때 현재일자가 평가일자 리스트의 첫번째 날짜보다 같거나 크면 step 1 반환
        if step == 0 and curdate >= datetime.strptime(datelist[0], "%Y-%m-%d"):
            step = 1
            return step
        # step 이 1 일 때 현재일자가 평가일자 리스트의 두번째 날짜보다 같거나 크면 step 2 반환
        elif step == 1 and curdate >= datetime.strptime(datelist[1], "%Y-%m-%d"):
            step = 2
            return step
        # step 이 2 일 때 현재일자가 평가일자 리스트의 세번째 날짜보다 같거나 크면 step 3 반환
        elif step == 2 and curdate >= datetime.strptime(datelist[2], "%Y-%m-%d"):
            step = 3
            return step
        # step 이 3 일 때 현재일자가 평가일자 리스트의 네번째 날짜보다 같거나 크면 step 4 반환
        elif step == 3 and curdate >= datetime.strptime(datelist[3], "%Y-%m-%d"):
            step = 4
            return step
        # step 이 4 일 때 현재일자가 평가일자 리스트의 다섯번째 날짜보다 같거나 크면 step 5 반환
        elif step == 4 and curdate >= datetime.strptime(datelist[4], "%Y-%m-%d"):
            step = 5
            return step
        # step 이 5 일 때 현재일자가 평가일자 리스트의 여섯번째 날짜보다 같거나 크면 step 6 반환
        elif step == 5 and curdate >= datetime.strptime(datelist[5], "%Y-%m-%d"):
            step = 6
            return step
        else:
            return step
    
    def Early_PaymentPrice(self, basevalue: float, step):
	# step 1 일 때 입수된 basevalue 에 0.95 로 조기상환 기준가 생성
        if step == 1:
            price = basevalue * 0.95
        # step 2 일 때 입수된 basevalue 에 0.90 로 조기상환 기준가 생성
        elif step == 2:
            price = basevalue * 0.90
        # step 3 일 때 입수된 basevalue 에 0.85 로 조기상환 기준가 생성
        elif step == 3:
            price = basevalue * 0.85
        # step 4 일 때 입수된 basevalue 에 0.80 로 조기상환 기준가 생성
        elif step == 4:
            price = basevalue * 0.80
        # step 5 일 때 입수된 basevalue 에 0.75 로 조기상환 기준가 생성
        elif step == 5:
            price = basevalue * 0.75
        # step 6 일 때 입수된 basevalue 에 0.65 로 조기상환 기준가 생성
        elif step == 6:
            price = basevalue * 0.65
        return price

    def Get_CurrentValue(self, discountrate: int, duration: int, futurevalue: int):
        # 미래가치 * exp(-할인율 * 투자기간(일수)/365 ) 
        currentvalue = futurevalue * np.exp(-discountrate*duration/365)
        # 현재가치 반환
        return currentvalue
    
    # Get_StochasticReturn 함수는 maxno로 시뮬레이션 횟수, rfr로 현재가치화 하는 금리, assetvol 로 
    # 기초자산 변동성, timeunit으로 시간단위, elsreturn으로 els 수익률, datelist로 영업일 list
    # contractdates로 계약 시작일, 종료일 리스트, terminationdates로 중도청산일자 list 등을 변수로 받는다.
    def Get_StochasticReturn(self, maxno: int, rfr: int, assetvol: int, timeunit: int, elsreturn: float, \
        datelist, contractdates: list, terminationdates: list):
        # 시나리오별 주가경로를 저장할 빈 dataframe을 만든다.
        totdf = pd.DataFrame()
        # 시나리오별 수익률, 종료일, 미래가치, 현재가치 등을 저장할 빈 dataframe를 만든다.
        asset_fin_result = []
        # 계약시작일자를 날짜형식으로 변수화
        contstartdate = datetime.strptime(contractdates[0], "%Y-%m-%d")
        for simulationno in range(1, maxno + 1):
            columns = ["DATE", f"SCENARIO{simulationno}"]
            columns_return = ["SCENARIO", "TERMINATIONDATE", "RETURN", "TOTVALUE", "CURRENTVALUE", "DURATION"]
            subdf = pd.DataFrame(columns=columns)
            # 하나의 시나리오 결과를 results list에 저장
            results = []
            # step 0부터 시작
            step = 0
            maxreturn = elsreturn
            for date in datelist:
                # 첫번째 평가일의 경우 자산가치 = 최초가치
                if date == datelist[0]:
                    curvalue = basevalue
                    result = [date, curvalue]
                    results.append(result)
                # 그 외의 경우
                else:
                    # GeometricBrownianMotion함수로 다음날의 기초자산 가치 계산
                    nextvalue = self.GeometricBrownianMotion(curvalue, rfr, assetvol, timeunit)
                    curvalue = nextvalue
                    # 투자기간 구하기
                    dur = (date - contstartdate).days
                    # 재평가일자에 해당되고, 마지막 이전의 재평가일자일 경우
                    if step!=5 and step != self.Early_PaymentDate(terminationdates, step, date):
                        step = self.Early_PaymentDate(terminationdates, step, date)
                        # 조기상환 조건충종할 경우
                        if curvalue >= self.Early_PaymentPrice(basevalue, step):
                            fv = maxreturn/12*step*6*basevalue+basevalue
                            pv = self.Get_CurrentValue(rfr, dur, fv)
                            # 원금 + 연 5.01% 수익 받고 해당 시뮬레이션 종료
                            asset_result = [f"SCENARIO{simulationno}", date, maxreturn, fv, pv, dur/365]
                            break
                    # 재평가일자에 해당되고 마지막 재평가일자일 경우
                    elif step==5 and step != self.Early_PaymentDate(terminationdates, step, date):
                        # 기초자산가치가 "초기가치 * 65%"보다 큰 경우
                        if curvalue >= self.Early_PaymentPrice(basevalue, step):
                            fv = maxreturn/12*step*6*basevalue+basevalue
                            pv = self.Get_CurrentValue(rfr, dur, fv)
                            # 원금 + 연 5.01% 수익 받고 해당 시뮬레이션 종료
                            asset_result = [f"SCENARIO{simulationno}", date, maxreturn, fv, pv, dur/365]
                            break
                        # 그 외의 경우(기초자산가치가 65% 보다 더 적은 경우)
                        else:
                            fv = maxreturn/12*step*6*basevalue + basevalue * (curvalue/basevalue)
                            pv = self.Get_CurrentValue(rfr, dur, fv)
                            # 원금손실 + 연 5.01% 수익 받고 해당 시뮬레이션 종료
                            asset_result = [f"SCENARIO{simulationno}", date, (fv/basevalue-1)*365/dur, fv, pv, dur/365]
                            break
                    # 재평가일자에 해당되지 않을 경우 시나리오list(results)에 주가경로 결과만 추가
                    else:
                        result = [date, curvalue]
                        results.append(result)
            # datelist를 거치면서 만들어진 results list를 dataframe 형식으로 변환
            subdf = pd.DataFrame(data=results, columns=columns)
            # 시뮬레이션 결과를 "DATE"컬럼을 기준으로 오른쪽컬럼에 붙이기
            if simulationno == 1:
                totdf = subdf
            else:
                totdf = pd.merge(totdf, subdf, on="DATE")
            # asset_fin_result에 시뮬레이션 결과 요약을 추가하기(row 추가) 
            asset_fin_result.append(asset_result)
        return_df = pd.DataFrame(data=asset_fin_result, columns=columns_return)  # 이게 최종 결과
        return totdf, return_df

In [4]:
if __name__ =="__main__":
    els = SNP500ELS001("^GSPC", "XNYS")
    contractdates = ["2023-11-27", "2026-12-04"]
    datadates = ["2007-06-01", "2009-06-30"]
    assetvol = els.Get_Variance(datadates[0], datadates[1])
    datelist = els.Get_TimeUnit(contractdates[0], contractdates[1])
    timeunit = len(datelist)
    basevalue = 100
    rfr = 0.0532
    elsreturn = 0.0501
    terminationdates = ["2024-06-05", "2024-12-04", "2025-06-04", "2025-12-04"\
                    , "2026-06-05", "2026-12-04"]
    trialno = 100
    
    totdf, return_df = els.Get_StochasticReturn(trialno, rfr, assetvol, timeunit,\
                                                 elsreturn, datelist, contractdates, terminationdates)
    meandf = pd.DataFrame(columns=["RETURN", "TOTVALUE", "CURRENTVALUE", "DURATION"])
    for column in meandf.columns:
        meandf.loc[0, column] = return_df[column].mean()
    print(meandf)

C:\Users\Dongjun\AppData\Local\Temp\ipykernel_25320\3237762215.py:18: FutureWarning: YF.download() has changed argument auto_adjust default to True
  asset_price = yf.download(self.ticker, start=startdate, end=enddate)
[*********************100%***********************]  1 of 1 completed


   RETURN TOTVALUE CURRENTVALUE  DURATION
0  0.0501  102.505    99.690728  0.523288


현재가치 99, duration 0.5으로 6개월  
고작 5%지급하니 현재가치 낮은 것  
기초자산 변동성 낮은 거 가져와서 조기상환  
좋은 상품은 아닐지도  